### Features selection

This kerenel explains how to select features using sklearn.
It can be imporved by prefitting the models or by cross-validating and aggregating feature importances.

The idea is to see how correlation, simple model, and GBT models will vote for features.

This kerenl produces a selection report "report.csv" which can be used to select top-k features and testing them on differnet model.

*This is a draft work, and will be improved regularly.*


In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# utilities
import os
import gc
import glob
import random
from datetime import datetime
from pathlib import Path



# helpers
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# selectors
from sklearn.feature_selection import SelectKBest, chi2,  RFE, SelectFromModel
from sklearn.model_selection import train_test_split

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# base
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone

# scoring
from sklearn import metrics


In [2]:
# notebook options
pd.set_option("display.max_columns", 100)
%matplotlib inline


path = "../input/tabular-playground-series-sep-2021/"
train_file = "train.csv"
test_file = "test.csv"

In [3]:
# Load the training data
train = pd.read_csv(f'{path}{os.sep}{train_file}', index_col=0)
test = pd.read_csv(f'{path}{os.sep}{test_file}', index_col=0)

# Preview the data
train.describe().T

,count,mean,std,min,25%,50%,75%,max
f1,942672.0,9.020086e-02,4.356374e-02,-1.499100e-01,7.022700e-02,9.013500e-02,1.165000e-01,4.151700e-01
f2,942729.0,3.459637e-01,1.462507e-01,-1.904400e-02,2.830500e-01,3.891000e-01,4.584500e-01,5.189900e-01
f3,942428.0,4.068744e+03,6.415829e+03,-9.421700e+03,4.184300e+02,1.279500e+03,4.444400e+03,3.954400e+04
f4,942359.0,2.012140e-01,2.125103e-01,-8.212200e-02,3.508650e-02,1.370000e-01,2.971000e-01,1.319900e+00
f5,942514.0,3.048693e-01,1.453425e-01,-6.989800e-03,2.405200e-01,3.277900e-01,4.128300e-01,5.547500e-01
...,...,...,...,...,...,...,...,...
f115,942360.0,1.208876e+00,1.149588e-01,9.052700e-01,1.146800e+00,1.177200e+00,1.242000e+00,1.886700e+00
f116,942330.0,4.276905e+16,6.732441e+16,-8.944400e+15,2.321100e+14,1.327500e+16,5.278700e+16,3.249900e+17
f117,942512.0,3.959205e+03,3.155992e+03,-4.152400e+02,1.306200e+03,3.228000e+03,6.137900e+03,1.315100e+04
f118,942707.0,5.592672e-01,4.084261e-01,-1.512400e-01,2.765600e-01,4.734400e-01,7.462100e-01,2.743600e+00


In [4]:
train.sample(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,...,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
931970,0.082187,0.20847,1758.0000,0.142120,0.260980,-0.164270,-46.171,505360.0,2.206800e+15,38947.0000,1.44100,-1.031400e+07,0.069469,-0.355750,0.292250,11.0100,7.2652,-0.047339,14.4740,2.876400,4763.2,2.0969,109.220,1.21520,654.71,9.585400e+11,9.281000e+10,2563700.0,0.003231,0.79919,-0.85538,110310.0,-58163.0,-0.001168,6.517000e+16,-30.275,454.68,1.4589,377.65,0.003775,48.16700,0.892000,6.1739,50.77400,0.000800,4.1493,-0.75457,5.2106,-0.59512,0.005398,...,0.001763,2.6183,86.609,2.525800e+15,-1.325600e+10,0.008801,8.7558,1112.00,3055.0,2.89960,0.117670,6.8941,1.995700e+09,69.2520,-242460.0,4.1301,3.469800e+08,9.20810,229.63000,6795.10,7.3429,0.46406,1209.300,27.4910,0.825480,37.32800,1083.300,1.00110,1.431900e+13,1.2497,0.29745,0.7074,1075800.0,66.0320,-0.89376,0.081167,0.107610,0.001363,6.089900e+09,0.26258,-7.8647,2.2242,-3.95370,-2.78360,-507.10,0.95674,1.727100e+17,82.115,0.54018,0
119154,0.082624,0.36603,359.8400,0.002339,0.404060,-0.354420,324.920,282510.0,2.379700e+14,-61.5300,1.27940,3.631300e+09,0.342370,3.304400,-0.048113,2.9638,6.6174,5.073800,8.3545,0.092024,19091.0,3.8031,115.030,-0.30021,357.14,1.961400e+11,1.424800e+11,3179300.0,-0.002070,8.35020,0.93834,155170.0,1059600.0,-0.001756,1.173900e+17,-91.072,578.83,2.7619,694.68,0.008806,1859.70000,-0.024263,5.8395,93.58500,0.001108,4.2845,0.41081,8.9023,-0.80674,NaN,...,0.003546,1.3223,1063.000,7.633500e+14,-1.424400e+10,-0.005391,6.8368,11412.00,4418.9,-3.76900,0.097579,2.4434,7.979800e+10,153.9200,5892400.0,-20.9610,1.552900e+09,-0.47591,533.12000,2037.20,5.0595,0.43075,-15.477,64.5360,0.867240,6.61050,64.880,0.99840,3.276800e+13,1.1294,1.00670,35.1870,8929.6,7.3088,-48.28700,0.224860,0.637500,0.039043,2.762300e+09,0.79379,-31.7430,NaN,2.03080,4.81830,-212.20,1.30090,8.977000e+16,6620.800,0.38311,0
540831,0.009450,0.42710,-223.6300,0.001698,0.302340,1.733900,1448.500,4952.1,4.657800e+12,-6.7692,-0.66454,8.103000e+06,0.315270,-0.076824,0.084798,359.0500,7.2103,0.016138,44.7400,238.930000,3359.8,2.1730,106.390,1.22540,324.31,2.293400e+12,-1.670200e+10,NaN,0.101650,NaN,-0.39430,-11426.0,550420.0,0.000968,1.416500e+14,-290.390,429.79,1.6579,2505.40,0.085687,177.32000,0.022521,NaN,131.66000,0.009607,4.0911,0.86269,6.0436,-1.21040,0.306840,...,0.000751,1.2315,482.980,1.394500e+13,5.983700e+12,-0.002065,7.7984,42882.00,16713.0,10.13800,0.498070,1.9966,9.932100e+10,305.0000,NaN,22.8340,-5.964800e+07,43.98200,173.99000,581.98,8.5218,0.49738,28076.000,126.3300,1.041400,2.26360,1042.300,0.99905,-1.494900e+10,3.5000,0.68582,15.7240,378690.0,-6.2524,-77.62900,0.329150,-0.000352,0.009691,8.111200e+08,0.24442,-13.1890,1.7518,132.08000,-0.59273,9641.10,1.29290,3.221700e+13,360.380,0.18716,0
835172,0.070643,0.43618,7.1147,0.266790,0.437500,-0.292730,103.570,472150.0,5.143000e+11,368.5400,-2.71190,2.341600e+08,0.265410,0.386880,0.043383,7.7990,8.9918,1.969700,10.9910,2.619300,371450.0,2.1084,75.677,1.32790,248.12,1.826400e+10,1.517900e+12,364300.0,0.010759,5.74620,0.88150,370820.0,201760.0,0.002853,2.970800e+16,2577.500,105.30,3.0762,1782.80,0.099744,983.87000,1.002800,6.0392,33.42800,-0.002706,3.9938,0.68893,6.9299,-0.97178,-0.001763,...,0.000645,2.3417,1272.700,1.174500e+15,-3.643600e+09,1.022500,4.8320,143.19,41297.0,-1.14660,0.076443,1.4751,1.815700e+11,5.9101,23712000.0,909.6000,6.827900e+08,2.40860,0.74928,7457.30,2.1992,0.40940,5106.900,7.4189,1.072500,19.43300,342.780,1.00090,3.650900e+13,1.2158,0.42662,16.1420,5007.9,75.4670,545.13000,0.187300,0.243050,0.024998,

In [5]:
# Separate target from features
y = train['claim']
X = train.drop(['claim'], axis=1)
X_test = test

In [6]:
# Preview features
X.head().T

id,0,1,2,3,4
f1,1.085900e-01,1.009000e-01,1.780300e-01,1.523600e-01,1.162300e-01
f2,4.313900e-03,2.996100e-01,-6.980000e-03,7.259100e-03,5.029000e-01
f3,-3.756600e+01,1.182200e+04,9.072700e+02,7.801000e+02,-1.091500e+02
f4,1.736400e-02,2.765000e-01,2.721400e-01,2.517900e-02,2.979100e-01
f5,2.891500e-01,4.597000e-01,4.594800e-01,5.194700e-01,3.449000e-01
...,...,...,...,...,...
f114,4.378800e+03,9.132300e+02,4.511900e+04,4.952400e+03,3.856500e+03
f115,1.209600e+00,1.246400e+00,1.176400e+00,1.178400e+00,1.483000e+00
f116,8.613400e+14,7.575100e+15,3.218100e+14,4.533000e+12,-8.991300e+12
f117,1.401000e+02,1.861000e+03,3.838200e+03,4.889100e+03,NaN


In [7]:
# identify columns
numerical_cols = list(X.select_dtypes(include=np.number).columns)
non_numeric_cols = list(X.select_dtypes(include=['object', 'bool']).columns)

In [8]:
print(f'We have {len(numerical_cols)} numeric and {len(non_numeric_cols)} non-numeric features')

We have 118 numeric and 0 non-numeric features


In [9]:
features = non_numeric_cols + numerical_cols

In [10]:
X.describe().T.style.bar(subset=['mean'], color='#20c8f2')\
                   .background_gradient(subset=['std'], cmap='YlGn')

,count,mean,std,min,25%,50%,75%,max
f1,942672.000000,0.090201,0.043564,-0.149910,0.070227,0.090135,0.116500,0.415170
f2,942729.000000,0.345964,0.146251,-0.019044,0.283050,0.389100,0.458450,0.518990
f3,942428.000000,4068.744207,6415.829440,-9421.700000,418.430000,1279.500000,4444.400000,39544.000000
f4,942359.000000,0.201214,0.212510,-0.082122,0.035086,0.137000,0.297100,1.319900
f5,942514.000000,0.304869,0.145343,-0.006990,0.240520,0.327790,0.412830,0.554750
f6,942398.000000,-0.071458,2.123777,-12.791000,-1.120700,-0.380110,0.921940,11.202000
f7,942415.000000,1620.843815,1276.281403,-224.800000,481.545000,1446.100000,2495.900000,5426.600000
f8,942546.000000,377164.164157,345432.472849,-29843.000000,91209.000000,289670.000000,560560.000000,1913700.000000
f9,942670.000000,1806053749440377.750000,2335204188640509.000000,-1153300000000000.000000,11531000000000.000000,504305000000000.000000,3103100000000000.000000,10424000000000000.000000
f10,942696.000000,5323.442367,10068.380032,-26404.000000,75.876750,1073.200000,5693.200000,85622.000000


In [11]:
print(f'Features: {list(X.columns)}')

Features: ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118']


### Feature Engineering

In [12]:
# new features
# https://www.kaggle.com/hiro5299834/tps-sep-2021-single-lgbm
X['n_missing'] = X[features].isna().sum(axis=1)
X_test['n_missing'] = X_test[features].isna().sum(axis=1)

X['std'] = X[features].std(axis=1)
X_test['std'] = X_test[features].std(axis=1)

features += ['n_missing', 'std']

# imputation
X[features] = X[features].fillna(X[features].mean())
X_test[features] = X_test[features].fillna(X_test[features].mean())



### Feature Selection
See here for a good description: 
https://mlwhiz.com/blog/2019/08/07/feature_selection/?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com

In [13]:
# user defined selector
# in this case correlation
class CorrelationSelector():
    def __init__(self, n_features, corr_fnc=np.corrcoef):
        self.corr_fnc = corr_fnc
        self.n_features = n_features
        self.support = []
        self.__name__ = "Correlation"
        
    def fit(self, X, y, use_abs=True):
        correlations = {}
        for col in X.columns:
            corr = self.corr_fnc(X[col], y)[0, 1]
            if np.isnan(corr):
                corr = 0
            
            correlations[col] = corr
        if use_abs:
            correlations = dict(zip(correlations.keys(), [abs(v) for v in correlations.values()]))
        
        # sort them by correlation
        correlations = {k: v for k, v in sorted(correlations.items(), key=lambda item: item[1])}
        selected_features = list(correlations.keys())[-self.n_features:]
        
        self.support = [True if feature in selected_features else False for feature in list(X.columns)]
    
    def get_support(self):
        return self.support        

In [14]:

class FeatureSelector():
    """ Process a dictionay of selectors and produces a dataframe report
    """
    def __init__(self, selectors=[None]):
        self.selectors = selectors
        self.report = {}
        
        
    def fit(self, X, y,  verbose=True):
        self.report["Feature"] = X.columns.tolist()
        for name, selector in selectors.items():
            print('-' * 50)
            print(f'Selector: {name}')
            print('-' * 50)
            selector["model"].fit(X, y, **selector["params"])
            self.report[name] = selector["model"].get_support()
            
    def get_report(self,
                   sort=True):
        df = pd.DataFrame(self.report)
        df['Total'] = df.iloc[:,1:].sum(axis=1)
        if sort:
            df = df.sort_values(['Total'], ascending=False)
        
        return df

In [15]:
# let us define some models,

lr_params = {
    'max_iter':1000
}

# from optuna studies here https://www.kaggle.com/mlanhenke/tps-09-simple-blend-stacking-xgb-lgbm-catb

xgb_params = dict(
    eval_metric='auc',
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    learning_rate=0.01187431306013263,
    n_estimators=10000,
    n_jobs=-1,
    use_label_encoder=False,
    objective='binary:logistic',
    tree_method='gpu_hist',
    gpu_id=0,
    predictor='gpu_predictor')

lgbm_params = {
    'metric' : 'auc',
    'objective' : 'binary',
    'device_type': 'gpu', 
    'n_estimators': 10000, 
    'learning_rate': 0.12230165751633416, 
    'num_leaves': 1400, 
    'max_depth': 8, 
    'min_child_samples': 3100, 
    'reg_alpha': 10, 
    'reg_lambda': 65, 
    'min_split_gain': 5.157818977461183, 
    'subsample': 0.5, 
    'subsample_freq': 1, 
    'colsample_bytree': 0.2,
    'n_jobs': 4
}

catb_params = {
    'eval_metric' : 'AUC',
    'depth' : 5,
    'grow_policy' : 'SymmetricTree',
    'l2_leaf_reg' : 3.0,
    'random_strength' : 1.0,
    'learning_rate' : 0.1,
    'iterations' : 10000,
    'loss_function' : 'CrossEntropy',
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0,
    'thread_count':4
}

lr = LogisticRegression(**lr_params)
xgb = XGBClassifier(**xgb_params)
lgbm = LGBMClassifier(**lgbm_params)
cat = CatBoostClassifier(**catb_params)

In [16]:
# create some selectors
n_features = 100 # 
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2,random_state=42)

# correlation
corr_selector = CorrelationSelector(n_features=n_features)

#  RFE logistic 
rfe_lr_selector = RFE(estimator=lr, n_features_to_select=n_features, step=10, verbose=5)
rfe_xgb_selector = RFE(estimator=xgb, n_features_to_select=n_features, step=10, verbose=5)
rfe_lgbm_selector = RFE(estimator=lgbm, n_features_to_select=n_features, step=10, verbose=5)
rfe_cat_selector = RFE(estimator=cat, n_features_to_select=n_features, step=10, verbose=5)


# SelectFromModel
from_model_lr_selector = SelectFromModel(lr, max_features=n_features)
from_model_xgb_selector = SelectFromModel(xgb, max_features=n_features)
from_model_lgbm_selector = SelectFromModel(lgbm, max_features=n_features)
from_model_cat_selector = SelectFromModel(cat, max_features=n_features)


# "params":{'eval_set':[(valid_X, valid_y)], 'early_stopping_rounds': 300, 'verbose': 1000}},

selectors = {
    "corr_selector": {"model": corr_selector, "params":{}},
    
    "rfe_xgb_selector": {"model": rfe_xgb_selector, "params":{}},
    "from_model_xgb_selector": {"model": from_model_xgb_selector, "params":{}},
    
    "rfe_lgbm_selector": {"model": rfe_lgbm_selector, "params":{}},
    "from_model_lgbm_selector": {"model": from_model_lgbm_selector, "params":{}},
    
    "rfe_cat_selector": {"model": rfe_cat_selector, "params":{}},
    "from_model_cat_selector": {"model": from_model_cat_selector, "params":{}},
    
    "rfe_lr_selector": {"model": rfe_lr_selector, "params":{}},
    "from_model_lr_selector": {"model": from_model_lr_selector, "params":{}}
}

In [17]:
# run the selection task
featruer_selector = FeatureSelector(selectors)
featruer_selector.fit(train_X, train_y)
report = featruer_selector.get_report()

--------------------------------------------------
Selector: corr_selector
--------------------------------------------------
--------------------------------------------------
Selector: rfe_xgb_selector
--------------------------------------------------
Fitting estimator with 120 features.
Fitting estimator with 110 features.
--------------------------------------------------
Selector: from_model_xgb_selector
--------------------------------------------------
--------------------------------------------------
Selector: rfe_lgbm_selector
--------------------------------------------------
Fitting estimator with 120 features.
Fitting estimator with 110 features.
--------------------------------------------------
Selector: from_model_lgbm_selector
--------------------------------------------------
--------------------------------------------------
Selector: rfe_cat_selector
--------------------------------------------------
Fitting estimator with 120 features.
Fitting estimator with 110 f

In [18]:
report.head(20)

,Feature,corr_selector,rfe_xgb_selector,from_model_xgb_selector,rfe_lgbm_selector,from_model_lgbm_selector,rfe_cat_selector,from_model_cat_selector,rfe_lr_selector,from_model_lr_selector,Total
118,n_missing,True,True,True,True,False,True,True,True,False,7
64,f65,True,True,False,True,True,True,True,True,False,7
41,f42,True,True,False,True,True,True,True,True,False,7
46,f47,True,True,False,True,True,True,True,True,False,7
0,f1,True,True,False,True,True,True,True,False,False,6
53,f54,True,True,False,True,True,True,False,True,False,6
30,f31,True,True,False,True,True,True,False,True,False,6
31,f32,True,True,False,True,True,True,False,True,False,6
109,f110,True,True,False,True,True,True,False,True,False,6
34,f35,True,True,False,True,False,True,False,True,True,6


In [19]:
report.to_csv('report.csv')

In [20]:
# simple vote-thresholding
min_votes = 2 # at least two models find this feature helpful
report.loc[(report['Total'] >= min_votes), 'Feature']

118    n_missing
64           f65
41           f42
46           f47
0             f1
         ...    
18           f19
57           f58
40           f41
58           f59
100         f101
Name: Feature, Length: 119, dtype: object